In [1]:
import pandas as pd
import numpy as np
from pyDOE2 import *
import csv
import time
import datetime

In [2]:
sign = fracfact('a b c abc ab ac bc')
# print("Signs for a, b, c, d, ab, ac, bc")
# print(sign)
# print("Signs for just a, b, c, and d")
grid = sign[:, :4]
# grid

In [3]:
factor_names = ['Approach', 'Model', 'Performance', 'Parallelism']
interaction_names = [[f'{factor_names[0]} x {factor_names[1]}',  # two different interactions because the pair are confounded
                      f'{factor_names[2]} x {factor_names[3]}'
                     ],
                     [f'{factor_names[0]} x {factor_names[2]}',
                      f'{factor_names[1]} x {factor_names[3]}'
                     ],
                     [f'{factor_names[1]} x {factor_names[2]}',
                      f'{factor_names[0]} x {factor_names[3]}'
                     ]]
all_names = factor_names + interaction_names
factor_interaction_labels = ['A', 'B', 'C', 'D', ['AB', 'CD'], ['AC', 'BD'], ['BC', 'AD']]
repetitions = 2

# Guessing, make sure to fix these values later
levels = {
    factor_names[0]: {
        'low': 'lazy',
        'high': 'predictive',
    },
    factor_names[1]: {
        'low': 'cifar',
        'high': 'fashion',
    },
    factor_names[2]: {
        'low': 'low-perform',
        'high': 'high-perform',
    },
    factor_names[3]: {
        'low': 'no-paral',
        'high': 'paral',
    }
}

runs_levels = [tuple([levels[fac]['low'] if r[i] == -1 else levels[fac]['high'] for i, fac in enumerate(factor_names)]) for r in grid]

# runs_levels

In [4]:
with open('sign-table.csv', 'w', newline='') as csvfile:
    sign_table_writer = csv.writer(csvfile, delimiter=',')
    sign_table_writer.writerow(['Run'] + [f for f in factor_names])
    for i in range(len(grid)):
        sign_table_writer.writerow([i+1] + ["+" if grid[i][j] == 1 else "-" for j in range(len(grid[i]))])

In [5]:
arrays = [
    [x[0] for x in runs_levels],
    [x[1] for x in runs_levels],
    [x[2] for x in runs_levels],
    [x[3] for x in runs_levels],
]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=factor_names)
# index

In [6]:
dim_sizes = {'runs': len(runs_levels), 'repetitions': repetitions}
df = pd.DataFrame(
#     np.abs(1 + np.random.randn(dim_sizes['runs'], dim_sizes['repetitions'])) * 60 * 4,
    np.zeros((dim_sizes['runs'], dim_sizes['repetitions'])),
    index=index
)

def to_seconds(string):
    t = string.split(':')
    return 3600 * int(t[0]) + 60 * int(t[1]) + float(t[2])

with open('data.csv', 'r') as data_file:
    datareader = csv.reader(data_file, delimiter=',')
    for r in datareader:
        ind = r[0].split('_')
        df.loc[ind[0], ind[1], ind[2], ind[3]] = [to_seconds(r[1]), to_seconds(r[2])]

# df = np.log(df)
df

,,,,0,1
Approach,Model,Performance,Parallelism,,
lazy,cifar,low-perform,no-paral,704.490920,683.337094
predictive,cifar,low-perform,paral,259.840318,265.190930
lazy,fashion,low-perform,paral,296.818196,274.589866
predictive,fashion,low-perform,no-paral,570.024752,506.233229
lazy,cifar,high-perform,paral,144.039178,143.047470
predictive,cifar,high-perform,no-paral,230.919316,217.241412
lazy,fashion,high-perform,no-paral,254.918687,234.813675
predictive,fashion,high-perform,paral,63.150942,75.521643


In [7]:
abs(df[0] + (np.random.random(1)*100)-50)

Approach    Model    Performance   Parallelism
lazy        cifar    low-perform   no-paral       753.289793
predictive  cifar    low-perform   paral          308.639192
lazy        fashion  low-perform   paral          345.617070
predictive  fashion  low-perform   no-paral       618.823625
lazy        cifar    high-perform  paral          192.838052
predictive  cifar    high-perform  no-paral       279.718189
lazy        fashion  high-perform  no-paral       303.717561
predictive  fashion  high-perform  paral          111.949816
Name: 0, dtype: float64

In [8]:
q = np.zeros((2**(4-1)))  # +1 for a spot for q0
# now add column of 1's for the 'I' column
sign_table = np.array([ np.concatenate((np.array([1.]), x)) for x in sign])
q, sign_table

(array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([[ 1., -1., -1., -1., -1.,  1.,  1.,  1.],
        [ 1.,  1., -1., -1.,  1., -1., -1.,  1.],
        [ 1., -1.,  1., -1.,  1., -1.,  1., -1.],
        [ 1.,  1.,  1., -1., -1.,  1., -1., -1.],
        [ 1., -1., -1.,  1.,  1.,  1., -1., -1.],
        [ 1.,  1., -1.,  1., -1., -1.,  1., -1.],
        [ 1., -1.,  1.,  1., -1., -1., -1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]]))

See box 18.1 for the following steps

$$q_j = \frac{1}{2^{k-1}} \sum_{i=1}^{k-1}S_{ij}\bar{y}_i$$

Where $k=4$ and $S_{ij}$ is the $(i,j)$ entry in the `sign_table`.

In [9]:
for j in range(len(q)):
    q[j] = (1/(2**(4-1))) * np.sum([sign_table[i][j] * df.iloc[i].mean() for i in range(2**(4-1))])
q

array([ 307.76110182,  -34.24578406,  -23.252228  , -137.30456128,
       -117.48628381,   53.46955163,   10.4975718 ,    9.89692442])

To calculate the sum of squares for each effect (as well as SS0/q0)

$$SSj = 2^{k-1}rq_j^2$$ for $j=0,1, 2,\dots,2^{k-1}-1$

In [10]:
SS = np.zeros((2**(4-1)))
for j in range(len(SS)):
    SS[j] = (2**(4-1)) * repetitions * (q[j] ** 2)
SS

array([1515470.33269475,   18764.37961161,    8650.65770784,
        301640.68076638,  220848.43012752,   45743.88722881,
          1763.18421891,    1567.1858077 ])

Now for $SSY$, $SST$, and $SSE$
$$SSY=\sum_{i=1}^{2^{k-1}}\sum_{j=1}^{r}y_{ij}^2$$
$$SST = SSY - SS0$$
$$SSE = SST - \sum_{j=1}^{2^{k-1}-1}SSj$$

In [11]:
SSY = np.sum([np.sum([df.iloc[i, j]**2 for j in range(repetitions)]) for i in range(2**(4-1))])
SST = SSY - SS[0]
SSE = SST - np.sum([SS[j] for j in range(1, 2**(4-1))])
SSY, SST, SSE

(2117341.180511269, 601870.8478165178, 2892.4423477489036)

% of `y`'s variation explained by `j`th effect
$$(SSj/SST) \cdot 100\%$$

In [12]:
var_expl = np.zeros((2**(4-1)))
for j in range(2**(4-1)):
    var_expl[j] = (SS[j] / SST) * 100
print(sum(var_expl[1:]))
[f"{all_names[i]}: {round(x, 2)}%" for i, x in enumerate(var_expl[1:])]  # just to pretty-print

99.51942474731213


['Approach: 3.12%',
 'Model: 1.44%',
 'Performance: 50.12%',
 'Parallelism: 36.69%',
 "['Approach x Model', 'Performance x Parallelism']: 7.6%",
 "['Approach x Performance', 'Model x Parallelism']: 0.29%",
 "['Model x Performance', 'Approach x Parallelism']: 0.26%"]

In [13]:
# % variation from error
var_expl_err = (SSE/SST)*100
print(f"% var from error: {round(var_expl_err, 2)}%")

% var from error: 0.48%


Standard deviation of errors:
$$s_e = \sqrt{\frac{SSE}{2^{k-1}(r-1)}}$$

In [14]:
s_e = np.sqrt(SSE/(2**(4-1)*(repetitions-1))) if repetitions > 1 else np.sqrt(SSE/(2**(4-1)))
s_e

19.01460737087708

In [15]:
s_qj = np.array([s_e / np.sqrt(2**(4-1)*repetitions)] * (2**(4-1)))
s_qj

array([4.75365184, 4.75365184, 4.75365184, 4.75365184, 4.75365184,
       4.75365184, 4.75365184, 4.75365184])

## Confidence Intervals
To find the confidence interval for effect $i$ the following equation is used:
$$q_i \pm t_{[1-\alpha/2; 2^{k-1}(r-1)]}s_{qi}$$

In [16]:
from scipy import stats

In [17]:
conf_interval = np.zeros_like((q))[1:]  # remove q0
conf = 0.95
dof = 2**(4-1)*(repetitions-1)
for i in range(len(conf_interval)):
    interval = stats.t.ppf((1 + conf)/2, dof) * s_qj[i+1]
    conf_interval[i] = interval
[(f"{all_names[i]}: {round(q[i+1] - interval, 2)}, {round(q[i+1] + interval, 2)}") for i, interval in enumerate(conf_interval)]

['Approach: -45.21, -23.28',
 'Model: -34.21, -12.29',
 'Performance: -148.27, -126.34',
 'Parallelism: -128.45, -106.52',
 "['Approach x Model', 'Performance x Parallelism']: 42.51, 64.43",
 "['Approach x Performance', 'Model x Parallelism']: -0.46, 21.46",
 "['Model x Performance', 'Approach x Parallelism']: -1.07, 20.86"]

## ANOVA

See table 22.5

In [18]:
MSE = s_e**2
# all degrees of freedoms of factors are 1 (since all levels are just 2), so MS is the same as SS
# just have to slice [1:] to get rid of the SS0 term
MS = np.copy(SS[1:])

F_vals = MS/MSE

p_vals = np.zeros_like(F_vals)
error_degree_of_freedom = 2**(4-1)*(repetitions-1)

for i in range(len(F_vals)):   
    p_vals[i] = 1-stats.f.cdf(F_vals[i], 1, error_degree_of_freedom)

print(f"MSE = {round(MSE, 2)}, dof for error = {error_degree_of_freedom}")
ANOVA = pd.DataFrame(np.stack((MS, F_vals, p_vals)).T, columns=["Mean Square", "F Computed", "p-values"])
ANOVA

MSE = 361.56, dof for error = 8


,Mean Square,F Computed,p-values
0,18764.379612,51.899059,9.208102e-05
1,8650.657708,23.926237,1.206751e-03
2,301640.680766,834.286446,2.233755e-09
3,220848.430128,610.828922,7.677040e-09
4,45743.887229,126.519755,3.504899e-06
5,1763.184219,4.876665,5.823364e-02
6,1567.185808,4.334567,7.089857e-02


Output ANOVA table

In [19]:
rounding = 2
with open('anova.csv', 'w', newline='') as csvfile:
    anovawriter = csv.writer(csvfile, delimiter=',')
    anovawriter.writerow(['comp', 'ss', 'pvar', 'df', 'ms', 'f', 'pval', 'confint'])
    
    for i in range(len(sign[0])):
#         row_color = "\\rowcolor{gray!8}\n" if i % 2 == 0 else ""
        row_color = "\\rowcolor{gray!8}" if i % 2 == 0 else ""
        
        p = f"{p_vals[i]:.2e}"
        if p_vals[i] > 10**(-2):
            p = round(p_vals[i], rounding+1)
        if type(factor_interaction_labels[i]) == list:
            prepend = "\multirow{ -2}{*}{"
            append = "}"
            anovawriter.writerow([row_color + f"{factor_interaction_labels[i][0]}: {all_names[i][0]}",
#                                   f'Counfounded with {factor_interaction_labels[i][0]}', '', '', '', '', '', '', ''
                                  '', '', '', '', '', '', '', ''
                                 ])
            anovawriter.writerow([row_color + f"{factor_interaction_labels[i][1]}: {all_names[i][1]}",
                                  prepend + str(round(SS[i+1], rounding)) + append,
                                  prepend + f"{round(var_expl[i+1], rounding)}\%" + append,
                                  prepend + str(1) + append,
                                  prepend + str(round(MS[i], rounding)) + append,
                                  prepend + str(round(F_vals[i], rounding)) + append,
                                  prepend + str(p) + append,
                                  prepend + f"{round(q[i+1] - interval, 2)}, {round(q[i+1] + interval, 2)}" + append
                                 ])
            
        else:
            anovawriter.writerow([row_color + f"{factor_interaction_labels[i]}: {all_names[i]}",
                                  round(SS[i+1], rounding),f"{round(var_expl[i+1], rounding)}\%",
                                  1, round(MS[i], rounding),
                                  round(F_vals[i], rounding), p,
                                  f"{round(q[i+1] - interval, 2)}, {round(q[i+1] + interval, 2)}"
                                 ])
        
    anovawriter.writerow(['Error', round(SSE, rounding), f"{round((SSE/SST)*100, rounding)}\%",
                          error_degree_of_freedom, round(MSE, rounding), '', '', ''])
    anovawriter.writerow(['\\rowcolor{gray!8}Total', round(SST, rounding), "100\%", 2**(4-1)*repetitions,'','','', ''])